In [1]:
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tensorboard_logger import configure, log_value
from model import RecurrentAttention
from torchvision import transforms
from torchvision import datasets
import torch.nn.functional as F
import matplotlib.pyplot as plt
from utils import AverageMeter
from config import get_config
from utils import plot_images
from trainer import Trainer
from tqdm import tqdm
import pandas as pd
import torchvision
import numpy as np
import random
import pickle
import shutil
import torch
import time

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

from utils import *

In [2]:
q = 4
with open("../../pooling/data/migration_data.json") as m:
    mig_data = json.load(m)
m.close()
mig_data = pd.DataFrame.from_dict(mig_data, orient = 'index').reset_index()
mig_data.columns = ['muni_id', 'num_migrants']
mig_data['class'] = pd.qcut(mig_data['num_migrants'], q = q, labels = [i for i in range(q)])
mig_data.head()

,muni_id,num_migrants,class
0,484001001,42055.0,3
1,484001002,4017.0,3
2,484001003,11992.0,3
3,484001004,762.0,2
4,484001005,7551.0,3


In [3]:
image_names = get_png_names("../../attn/data/MEX/")
y_class, y_mig = get_ys(image_names, mig_data)

In [4]:
train_num = int(25 * .70)
train_indices = random.sample(range(0, 25), train_num)
val_indices = [i for i in range(0, 25) if i not in train_indices]

batch_size = 1
train = [(torchvision.transforms.functional.adjust_brightness(load_inputs(image_names[i]), brightness_factor = 2).squeeze(), y_class[i], y_mig[i]) for i in train_indices]
val = [(torchvision.transforms.functional.adjust_brightness(load_inputs(image_names[i]), brightness_factor = 2).squeeze(), y_class[i], y_mig[i]) for i in val_indices]
train_dl = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = True)
val_dl = torch.utils.data.DataLoader(val, batch_size = batch_size, shuffle = True)

In [5]:
print("Num training: ", len(train_dl))
print("Num validation: ", len(val_dl))

Num training:  17
Num validation:  8


In [6]:
0.5 * ((torch.tensor([.8]) + 1.0) * 2000)

tensor([1800.])

In [7]:
0.5 * ((torch.tensor([.2]) + 1.0) * 4000)

tensor([2400.])

In [8]:
H, W = torch.tensor([1,1])

In [9]:
W

tensor(1)

In [10]:
# lstm1 = torch.nn.LSTM(input_size = 256,
#                                    hidden_size = 128,
#                                    num_layers = 2,
#                                    bias = True,
#                                    batch_first = True)

# gn = torch.rand(1,4,256)
# h0 = torch.rand(2,1,128)
# c0 = torch.rand(2,1,128)

# out, (hn, cn) = lstm1(gn, (h0, c0))

# print(hn.shape)

In [11]:
config, unparsed = get_config()
trainer = Trainer(config, (train_dl, val_dl))

In [12]:
trainer.train()

0.1s - loss: 1561.275 - acc: 0.000 - mae: 1559.865:   0%|          | 0/17 [00:00<?, ?it/s]


[*] Train on 17 samples, validate on 8 samples

Epoch: 1/200 - LR: 0.000300


1.1s - loss: 43.032 - acc: 100.000 - mae: 40.757: 100%|██████████| 17/17 [00:01<00:00, 15.87it/s]    


train loss: 2599.779 - train acc: 23.529 - val loss: 5979.005 - val acc: 37.500 - val err: 62.500- train mae: 2598.109 - val mae: 5977.247 [*]


0.1s - loss: 22.018 - acc: 100.000 - mae: 19.749:   0%|          | 0/17 [00:00<?, ?it/s]

...

train loss: 2599.650 - train acc: 35.294 - val loss: 5978.816 - val acc: 37.500 - val err: 62.500- train mae: 2597.911 - val mae: 5977.078 [*]


0.0s - loss: 63.508 - acc: 100.000 - mae: 61.582:   0%|          | 0/17 [00:00<?, ?it/s]


Epoch: 4/200 - LR: 0.000300


0.0s - loss: 63.508 - acc: 100.000 - mae: 61.582:   6%|▌         | 1/17 [00:00<00:03,  4.21it/s]


KeyboardInterrupt: 

In [ ]:
trainer.model

In [ ]:
# for n,p in trainer.model.named_parameters():
#     print(n)

In [ ]:
        new_loc = torch.distributions.Normal(mu, self.std).rsample()
